In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import requests
from bs4 import BeautifulSoup
from langchain_text_splitters import HTMLHeaderTextSplitter

responses = []
responses.append(requests.get('https://scikit-learn.org/stable/modules/linear_model.html').text)
responses.append(requests.get('https://scikit-learn.org/stable/modules/lda_qda.html').text)

#soup = BeautifulSoup(response.text, 'html.parser')


In [8]:
headers = [
    ('h1', 'Page'),  # Adicionar link
    ('h2', 'Section'),
    ('h3', 'Sub Section'),
    ('h4', 'Sub Section'),
]

splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers,
)

documents = []

for r in responses:
    documents.append(splitter.split_text(r))

In [2]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

c:\Users\Windows11\AppData\Local\pypoetry\Cache\virtualenvs\langchain-llm-9c8IAL59-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
flat_documents = []
for html_split in documents:  # Desempacota as listas
    for chunk in html_split:
        flat_documents.append(chunk)

db = Chroma.from_documents(
    flat_documents, embeddings
)  

In [25]:
retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={"k": 5, 'score_threshold': 0.1}
)

In [28]:
prompt = ChatPromptTemplate.from_template("""
Responda a pergunta em português com base no contexto fornecido abaixo.
Inclua a fonte da infomação na responsta, diga o nome da página e a subseção da informação.
                                           
Contexto:                                   
{context}

Entrada do usuário:
{question}
""")


llm = OpenAI(model="gpt-4o-mini")

rag_chain = ( 
    {"context": retriever, "question": RunnableLambda(lambda x: x)}
    | prompt
    | llm
)

response = rag_chain.invoke("Quando ultilizar Lasso? Ele está relacionado a LDA?")

In [30]:
print(response)

 
Resposta:
O Lasso é utilizado quando se deseja estimar coeficientes esparsos em modelos lineares, sendo especialmente útil em contextos onde há muitas características, pois tende a preferir soluções com menos coeficientes não nulos, o que pode ajudar na seleção de características. Esta abordagem é fundamental no campo da compressão de dados. O Lasso e suas variantes podem ser aplicados em situações onde se quer evitar sobreajuste e melhorar a interpretação dos modelos. 

O Lasso não está diretamente relacionado à Análise Discriminante Linear (LDA). Enquanto o Lasso é um método de regularização em modelos lineares, o LDA é um classificador que busca maximizar a separação entre diferentes classes. 

Fonte: Página "Linear Models", Sub seção "Lasso".
